In [1]:
from sequential_tasks import TemporalOrderExp6aSequence

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [2]:
class SimpleRNN(nn.Module):

    def __init__(self, input_size, rnn_hidden_size, output_size):

        super(SimpleRNN, self).__init__()
        self.rnn = torch.nn.RNN(input_size, rnn_hidden_size, num_layers=1, nonlinearity='relu', batch_first=True)
        #self.h_0 = self.initialize_hidden(rnn_hidden_size)
        self.linear = torch.nn.Linear(rnn_hidden_size, output_size) 

    def forward(self, x):
        #x = x.unsqueeze(0)
        #self.rnn.flatten_parameters()
        #out, self.h_0 = self.rnn(x, self.h_0)
        out, _ = self.rnn(x)
        out = self.linear(out)
        return F.log_softmax(out, dim=1)

    #def initialize_hidden(self, rnn_hidden_size):
    #    return Variable(torch.randn(2, 1, rnn_hidden_size), requires_grad=True)

In [3]:
def exp6a_experiment(settings):
    train_data_gen = TemporalOrderExp6aSequence.get_predefined_generator(
        settings['difficulty'],
        settings['batch_size'])

    model = SimpleRNN(train_data_gen.n_symbols, settings['h_units'], train_data_gen.n_classes)

    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001)

    epochs = settings['max_epochs']
    log_interval = settings['log_interval']

    epoch = 0
    while epoch < epochs:
        predictions = []
        truth_values = []
   
        for batch_idx, (xs, ys) in enumerate(train_data_gen):
            xs, ys = torch.from_numpy(xs).float(), torch.from_numpy(ys).long()

            y_pred = model(xs)
            loss = criterion(y_pred, ys)
            optimizer.zero_grad()
            loss.backward()
            #nn.utils.clip_grad_norm(model.parameters(), 0.5)
            optimizer.step()

            predictions.append(y_pred.cpu().data.numpy().ravel())
            truth_values.append(ys.cpu().data.numpy().ravel())

            if batch_idx % log_interval == 0:
                print('Train Epoch: {}, mini-batch {} of {}, training loss: {:.6f}'.format(
                    epoch, batch_idx, len(train_data_gen), loss.item()))

        epoch += 1
    
    # testing
    test_data_gen = TemporalOrderExp6aSequence.get_predefined_generator(
        settings['difficulty'],
        settings['batch_size'])

    correct = 0
    with torch.no_grad():
        for i, (xs, ys) in enumerate(test_data_gen):
            xs, ys = torch.from_numpy(xs).float(), torch.from_numpy(ys).long()
            output = model(xs)
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(ys.view_as(pred)).sum().item()

    test_accuracy = float(correct) / len(test_data_gen)
    print('\nAccuracy: {}'.format(test_accuracy))
    
    return test_accuracy

In [4]:
# experiments settings
params = {
    "difficulty": TemporalOrderExp6aSequence.DifficultyLevel.EASY,
    "batch_size": 32,
    "h_units": 4,
    "max_epochs": 30,
    "log_interval": 10
}

acc = exp6a_experiment(params)

Train Epoch: 0, mini-batch 0 of 31, training loss: 2.223053
Train Epoch: 0, mini-batch 10 of 31, training loss: 2.174639
Train Epoch: 0, mini-batch 20 of 31, training loss: 2.136809
Train Epoch: 0, mini-batch 30 of 31, training loss: 2.114726
Train Epoch: 0, mini-batch 40 of 31, training loss: 2.083107
Train Epoch: 0, mini-batch 50 of 31, training loss: 2.044508
Train Epoch: 0, mini-batch 60 of 31, training loss: 2.015424
Train Epoch: 0, mini-batch 70 of 31, training loss: 1.956085
Train Epoch: 0, mini-batch 80 of 31, training loss: 1.902780
Train Epoch: 0, mini-batch 90 of 31, training loss: 1.773027
Train Epoch: 0, mini-batch 100 of 31, training loss: 1.618411
Train Epoch: 0, mini-batch 110 of 31, training loss: 1.461884
Train Epoch: 0, mini-batch 120 of 31, training loss: 1.294207
Train Epoch: 0, mini-batch 130 of 31, training loss: 1.150648
Train Epoch: 0, mini-batch 140 of 31, training loss: 1.032733
Train Epoch: 0, mini-batch 150 of 31, training loss: 0.936927
Train Epoch: 0, min

Train Epoch: 0, mini-batch 1360 of 31, training loss: 0.564101
Train Epoch: 0, mini-batch 1370 of 31, training loss: 0.562060
Train Epoch: 0, mini-batch 1380 of 31, training loss: 0.561283
Train Epoch: 0, mini-batch 1390 of 31, training loss: 0.563563
Train Epoch: 0, mini-batch 1400 of 31, training loss: 0.564927
Train Epoch: 0, mini-batch 1410 of 31, training loss: 0.561556
Train Epoch: 0, mini-batch 1420 of 31, training loss: 0.563803
Train Epoch: 0, mini-batch 1430 of 31, training loss: 0.563216
Train Epoch: 0, mini-batch 1440 of 31, training loss: 0.563819
Train Epoch: 0, mini-batch 1450 of 31, training loss: 0.562274
Train Epoch: 0, mini-batch 1460 of 31, training loss: 0.563354
Train Epoch: 0, mini-batch 1470 of 31, training loss: 0.564777
Train Epoch: 0, mini-batch 1480 of 31, training loss: 0.562981
Train Epoch: 0, mini-batch 1490 of 31, training loss: 0.562601
Train Epoch: 0, mini-batch 1500 of 31, training loss: 0.563900
Train Epoch: 0, mini-batch 1510 of 31, training loss: 0

Train Epoch: 0, mini-batch 2670 of 31, training loss: 0.563423
Train Epoch: 0, mini-batch 2680 of 31, training loss: 0.560111
Train Epoch: 0, mini-batch 2690 of 31, training loss: 0.568801
Train Epoch: 0, mini-batch 2700 of 31, training loss: 0.564765
Train Epoch: 0, mini-batch 2710 of 31, training loss: 0.561211
Train Epoch: 0, mini-batch 2720 of 31, training loss: 0.561496
Train Epoch: 0, mini-batch 2730 of 31, training loss: 0.561914
Train Epoch: 0, mini-batch 2740 of 31, training loss: 0.561319
Train Epoch: 0, mini-batch 2750 of 31, training loss: 0.563060
Train Epoch: 0, mini-batch 2760 of 31, training loss: 0.563344
Train Epoch: 0, mini-batch 2770 of 31, training loss: 0.563499
Train Epoch: 0, mini-batch 2780 of 31, training loss: 0.561237
Train Epoch: 0, mini-batch 2790 of 31, training loss: 0.562534
Train Epoch: 0, mini-batch 2800 of 31, training loss: 0.562207
Train Epoch: 0, mini-batch 2810 of 31, training loss: 0.560933
Train Epoch: 0, mini-batch 2820 of 31, training loss: 0

Train Epoch: 0, mini-batch 3980 of 31, training loss: 0.560223
Train Epoch: 0, mini-batch 3990 of 31, training loss: 0.561195
Train Epoch: 0, mini-batch 4000 of 31, training loss: 0.561496
Train Epoch: 0, mini-batch 4010 of 31, training loss: 0.564278
Train Epoch: 0, mini-batch 4020 of 31, training loss: 0.564955
Train Epoch: 0, mini-batch 4030 of 31, training loss: 0.564873
Train Epoch: 0, mini-batch 4040 of 31, training loss: 0.562009
Train Epoch: 0, mini-batch 4050 of 31, training loss: 0.560021
Train Epoch: 0, mini-batch 4060 of 31, training loss: 0.557637
Train Epoch: 0, mini-batch 4070 of 31, training loss: 0.562466
Train Epoch: 0, mini-batch 4080 of 31, training loss: 0.563855
Train Epoch: 0, mini-batch 4090 of 31, training loss: 0.562744
Train Epoch: 0, mini-batch 4100 of 31, training loss: 0.561262
Train Epoch: 0, mini-batch 4110 of 31, training loss: 0.560494
Train Epoch: 0, mini-batch 4120 of 31, training loss: 0.566025
Train Epoch: 0, mini-batch 4130 of 31, training loss: 0

Train Epoch: 0, mini-batch 5310 of 31, training loss: 0.561818
Train Epoch: 0, mini-batch 5320 of 31, training loss: 0.561537
Train Epoch: 0, mini-batch 5330 of 31, training loss: 0.561457
Train Epoch: 0, mini-batch 5340 of 31, training loss: 0.562035
Train Epoch: 0, mini-batch 5350 of 31, training loss: 0.562940
Train Epoch: 0, mini-batch 5360 of 31, training loss: 0.561329
Train Epoch: 0, mini-batch 5370 of 31, training loss: 0.559641
Train Epoch: 0, mini-batch 5380 of 31, training loss: 0.562144
Train Epoch: 0, mini-batch 5390 of 31, training loss: 0.562394
Train Epoch: 0, mini-batch 5400 of 31, training loss: 0.560663
Train Epoch: 0, mini-batch 5410 of 31, training loss: 0.561620
Train Epoch: 0, mini-batch 5420 of 31, training loss: 0.565256
Train Epoch: 0, mini-batch 5430 of 31, training loss: 0.564751
Train Epoch: 0, mini-batch 5440 of 31, training loss: 0.559852
Train Epoch: 0, mini-batch 5450 of 31, training loss: 0.566071
Train Epoch: 0, mini-batch 5460 of 31, training loss: 0

...

Train Epoch: 0, mini-batch 30360 of 31, training loss: 0.559908
Train Epoch: 0, mini-batch 30370 of 31, training loss: 0.562644
Train Epoch: 0, mini-batch 30380 of 31, training loss: 0.561080
Train Epoch: 0, mini-batch 30390 of 31, training loss: 0.565258
Train Epoch: 0, mini-batch 30400 of 31, training loss: 0.563976
Train Epoch: 0, mini-batch 30410 of 31, training loss: 0.562737
Train Epoch: 0, mini-batch 30420 of 31, training loss: 0.561400
Train Epoch: 0, mini-batch 30430 of 31, training loss: 0.561978
Train Epoch: 0, mini-batch 30440 of 31, training loss: 0.561217
Train Epoch: 0, mini-batch 30450 of 31, training loss: 0.562777
Train Epoch: 0, mini-batch 30460 of 31, training loss: 0.561701
Train Epoch: 0, mini-batch 30470 of 31, training loss: 0.561834
Train Epoch: 0, mini-batch 30480 of 31, training loss: 0.562559
Train Epoch: 0, mini-batch 30490 of 31, training loss: 0.562371
Train Epoch: 0, mini-batch 30500 of 31, training loss: 0.563052
Train Epoch: 0, mini-batch 30510 of 31, 

Train Epoch: 0, mini-batch 31680 of 31, training loss: 0.562926
Train Epoch: 0, mini-batch 31690 of 31, training loss: 0.564243
Train Epoch: 0, mini-batch 31700 of 31, training loss: 0.563769
Train Epoch: 0, mini-batch 31710 of 31, training loss: 0.561688
Train Epoch: 0, mini-batch 31720 of 31, training loss: 0.562754
Train Epoch: 0, mini-batch 31730 of 31, training loss: 0.563013
Train Epoch: 0, mini-batch 31740 of 31, training loss: 0.561404
Train Epoch: 0, mini-batch 31750 of 31, training loss: 0.562542
Train Epoch: 0, mini-batch 31760 of 31, training loss: 0.562241
Train Epoch: 0, mini-batch 31770 of 31, training loss: 0.563554
Train Epoch: 0, mini-batch 31780 of 31, training loss: 0.562730
Train Epoch: 0, mini-batch 31790 of 31, training loss: 0.562538
Train Epoch: 0, mini-batch 31800 of 31, training loss: 0.563599
Train Epoch: 0, mini-batch 31810 of 31, training loss: 0.560992
Train Epoch: 0, mini-batch 31820 of 31, training loss: 0.562521
Train Epoch: 0, mini-batch 31830 of 31, 

Train Epoch: 0, mini-batch 33000 of 31, training loss: 0.560288
Train Epoch: 0, mini-batch 33010 of 31, training loss: 0.564290
Train Epoch: 0, mini-batch 33020 of 31, training loss: 0.559521
Train Epoch: 0, mini-batch 33030 of 31, training loss: 0.562920
Train Epoch: 0, mini-batch 33040 of 31, training loss: 0.560678
Train Epoch: 0, mini-batch 33050 of 31, training loss: 0.561277
Train Epoch: 0, mini-batch 33060 of 31, training loss: 0.559191
Train Epoch: 0, mini-batch 33070 of 31, training loss: 0.563562
Train Epoch: 0, mini-batch 33080 of 31, training loss: 0.560737
Train Epoch: 0, mini-batch 33090 of 31, training loss: 0.564203
Train Epoch: 0, mini-batch 33100 of 31, training loss: 0.560770
Train Epoch: 0, mini-batch 33110 of 31, training loss: 0.562448
Train Epoch: 0, mini-batch 33120 of 31, training loss: 0.561523
Train Epoch: 0, mini-batch 33130 of 31, training loss: 0.558756
Train Epoch: 0, mini-batch 33140 of 31, training loss: 0.565608
Train Epoch: 0, mini-batch 33150 of 31, 

Train Epoch: 0, mini-batch 34300 of 31, training loss: 0.561946
Train Epoch: 0, mini-batch 34310 of 31, training loss: 0.563246
Train Epoch: 0, mini-batch 34320 of 31, training loss: 0.563768
Train Epoch: 0, mini-batch 34330 of 31, training loss: 0.562655
Train Epoch: 0, mini-batch 34340 of 31, training loss: 0.562917
Train Epoch: 0, mini-batch 34350 of 31, training loss: 0.563216
Train Epoch: 0, mini-batch 34360 of 31, training loss: 0.562174
Train Epoch: 0, mini-batch 34370 of 31, training loss: 0.561127
Train Epoch: 0, mini-batch 34380 of 31, training loss: 0.560015
Train Epoch: 0, mini-batch 34390 of 31, training loss: 0.563529
Train Epoch: 0, mini-batch 34400 of 31, training loss: 0.562895
Train Epoch: 0, mini-batch 34410 of 31, training loss: 0.561903
Train Epoch: 0, mini-batch 34420 of 31, training loss: 0.562056
Train Epoch: 0, mini-batch 34430 of 31, training loss: 0.562076
Train Epoch: 0, mini-batch 34440 of 31, training loss: 0.563070
Train Epoch: 0, mini-batch 34450 of 31, 

KeyboardInterrupt: 

In [ ]:
print('acc = {:.2f}%.'.format(acc))